<div align="center">

## UNIVERSIDAD TECNOLÓGICA DE LA MIXTECA

### Análisis de Textos para la Ingeniería de Requisitos

### Proyecto 1.1. Obtención y preparación de datos textuales para análisis

---

**Maestría en Ingeniería de Software**  

**Alumna:** Ing. Viviana Isabel Salazar Vasquez  

**Profesor:** Dr. Christian Eduardo Millán Hernández  

---

Huajuapan de León, Oaxaca  
25 de Septiembre 2025

</div>


### Fuente y justificación de la elección

Los artículos científicos fueron obtenidos de **arXiv**, una base de datos abierta de publicaciones académicas. La elección de trabajar con artículos científicos se debe a que su contenido y estructura de información será de gran ayuda para el desarrollo de mi **tema de tesis**, especialmente en el área de **Ingeniería de Requisitos y PLN**.

### Pasos realizados para recolectar los datos

1. Se identificaron los artículos relevantes en arXiv usando las siguientes cadenas de búsqueda:  
   - `"nlp" AND "requirements engineering" AND "modeling"`  
   - `"use case modeling requirements using nlp"`  

2. Se descargaron los PDFs de los artículos mediante la ayuda de la API de arXiv.

3. Para cada artículo se obtuvieron los metadatos de la siguiente manera:  
   - **Primero**: a través de la API de arXiv usando la librería `arxiv`.  
   - **Segundo**: con la librería `habanero` (Crossref) en caso de que arXiv no devolviera toda la información.  
   - **Tercero**: se extrajo información directamente del PDF cuando los metadatos anteriores no estaban disponibles (título, autores, fecha).

### Herramientas o librerías utilizadas

- `arxiv` para la búsqueda y descarga de artículos científicos.  
- `habanero` para obtener metadatos complementarios desde Crossref.  
- `PyMuPDF (fitz)` para manipulación de PDFs y extracción de texto.  
- `re` y `html` de Python para limpieza de texto.  
- `pandas` para organizar los datos en DataFrames.  
- `tqdm` para visualizar el progreso de la recolección de datos.


Importe de librerías:

In [ ]:
import re
import html
import fitz 
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import arxiv
from habanero import Crossref

c:\Users\salaz\miniconda3\envs\analisis312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Directorios de archivos

In [ ]:
# Directorios de archivos pdf's y html
pdf_dir = Path("data/pdfs")
html_dir = Path("data/htmls")
html_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
#Instancia de Crossref
cr = Crossref()

Generación de los HTML's

In [ ]:
def pdf_to_html(pdf_file, out_dir=html_dir):
    """Convierte un archivo PDF a HTML y lo guarda en el directorio especificado."""
    doc = fitz.open(pdf_file)
    html_path = out_dir / f"{pdf_file.stem}.html"
    with open(html_path, "w", encoding="utf-8") as f:
        for page in doc:
            f.write(page.get_text("html"))
    return html_path

Método para limpiar los datos
- Los datos se limpian con la aplicación de regex

In [ ]:
def extract_text_from_html(html_file): 
    """Extrae y limpia el texto de un archivo HTML."""
    with open(html_file, encoding="utf-8") as f: 
        html_content = f.read() 
        # quitar etiquetas 
        text = re.sub(r"<[^>]+>", " ", html_content) 
        # decodificar entidades HTML (como &#xe9; → é, &#xa0; → espacio) 
        text = html.unescape(text) 
        # limpiar espacios múltiples 
        text = re.sub(r"\s+", " ", text) 
    return text.strip()

Métodos para la obtención de los metadatos (Título, autor, fecha y DOI):

In [7]:
def get_metadata_arxiv(arxiv_id):
    """Obtiene metadatos desde la API de arXiv (incluye DOI si existe)."""
    try:
        clean_id = re.sub(r"v\d+$", "", arxiv_id)
        search = arxiv.Search(id_list=[clean_id])
        result = next(search.results())
        return {
            "titulo": result.title.strip(),
            "autor": ", ".join([a.name for a in result.authors]),
            "fecha": result.published.strftime("%Y-%m-%d"),
            "arxiv_id": result.entry_id.split("/")[-1],
            "doi": result.doi if result.doi else ""
        }
    except Exception:
        return {"titulo": "", "autor": "", "fecha": "", "arxiv_id": arxiv_id, "doi": ""}

In [ ]:
def get_title_from_pdf(pdf_file):
    """Extrae el título del primer página de un archivo PDF."""
    try:
        doc = fitz.open(pdf_file)
        first_page = doc[0].get_text("text").split("\n")
        candidates = [line.strip() for line in first_page if len(line.strip()) > 10]
        return candidates[0] if candidates else ""
    except Exception:
        return ""

In [ ]:
def get_metadata_crossref(doi):
    """Obtiene metadatos desde la API de Crossref usando un DOI."""
    try:
        res = cr.works(ids=doi)
        item = res["message"]
        autores = ", ".join([f"{a.get('given', '')} {a.get('family', '')}".strip() for a in item.get("author", [])])
        fecha = ""
        if "published-print" in item:
            fecha = "-".join(str(x) for x in item["published-print"]["date-parts"][0])
        elif "published-online" in item:
            fecha = "-".join(str(x) for x in item["published-online"]["date-parts"][0])
        return autores, fecha
    except Exception:
        return "", ""


In [10]:
def get_doi_crossref(title):
    """Busca DOI en Crossref si arXiv no lo devuelve."""
    try:
        res = cr.works(query_title=title, limit=1)
        if res["message"]["items"]:
            item = res["message"]["items"][0]
            return item.get("DOI", ""), "crossref"
    except Exception:
        pass
    return "", ""


In [11]:
registros = []

Obtención de los metadatos para cada PDF
- En esta parte se aplican todos los método anteriormente establecidos, limpiando los datos y obteniendo los metadatos con ayuda de las APIs.

In [ ]:
for idx, pdf_file in enumerate(tqdm(list(pdf_dir.glob("*.pdf")), desc="Procesando PDFs"), start=1):
    """Realiza la obtención de metadatos y texto completo para cada PDF."""
    try:
        arxiv_id = pdf_file.stem

        # 1. Metadatos desde arXiv
        meta_arxiv = get_metadata_arxiv(arxiv_id)

        # 2. Fallback: título desde PDF si falta
        if not meta_arxiv["titulo"]:
            meta_arxiv["titulo"] = get_title_from_pdf(pdf_file)

        # 3. DOI (arXiv o Crossref)
        doi = meta_arxiv.get("doi", "")
        if not doi and meta_arxiv["titulo"]:
            doi, _ = get_doi_crossref(meta_arxiv["titulo"])
            meta_arxiv["doi"] = doi

        # 4. Autor y fecha (solo si no hay en arXiv y hay DOI de Crossref)
        if (not meta_arxiv.get("autor") or not meta_arxiv.get("fecha")) and doi:
            try:
                res = cr.works(ids=doi)
                item = res["message"]
                # Autor
                autores = ", ".join([
                    f"{a.get('given', '').strip()} {a.get('family', '').strip()}".strip() 
                    for a in item.get("author", [])
                ])
                meta_arxiv["autor"] = autores
                # Fecha
                fecha = ""
                if "published-print" in item:
                    fecha = "-".join(str(x) for x in item["published-print"]["date-parts"][0])
                elif "published-online" in item:
                    fecha = "-".join(str(x) for x in item["published-online"]["date-parts"][0])
                meta_arxiv["fecha"] = fecha
            except Exception:
                meta_arxiv["autor"] = meta_arxiv.get("autor", "")
                meta_arxiv["fecha"] = meta_arxiv.get("fecha", "")

        # 5. Texto completo (opcional)
        try:
            html_file = pdf_to_html(pdf_file)
            text = extract_text_from_html(html_file)
        except Exception:
            text = ""

        # 6. Registro
        registro = {
            "id": idx,
            "fuente": "pdf",
            "arxiv_id": meta_arxiv.get("arxiv_id", arxiv_id),
            "autor": meta_arxiv.get("autor", ""),
            "fecha": meta_arxiv.get("fecha", ""),
            "texto": text,
            "longitud": len(text.split()) if text else 0,
            "titulo": meta_arxiv.get("titulo", ""),
            "doi": meta_arxiv.get("doi", ""),
        }
        registros.append(registro)

    except Exception as e:
        print(f"Error con {pdf_file}: {e}")



Procesando PDFs:   0%|          | 0/200 [00:00<?, ?it/s]C:\Users\salaz\AppData\Local\Temp\ipykernel_7432\4267842114.py:6: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  result = next(search.results())
Procesando PDFs:  96%|█████████▋| 193/200 [25:40<01:25, 12.20s/it]

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object is not a stream

MuPDF error: format error: object out of range (539 0 R); xref size 512

MuPDF error: format error: object out of range (539 0 R); xref size 512

MuPDF error: format error: object out of range (539 0 R); xref size 512

MuPDF error: format error: object out of range (540 0 R); xref size 512

MuPDF error: format error: object out of range (540 0 R); xref size 512

MuPDF error: format error: object out of range (540 0 R); xref size 512

MuPDF error: format error: object out of range (541 0 R); xref size 512

MuPDF error: format error: object out of range (541 0 R); xref size 512

MuPDF error: format error: object out of range (541 0 R); xref size 512

MuPDF error: format error: object out

Procesando PDFs: 100%|██████████| 200/200 [26:53<00:00,  8.07s/it]


Cración del DataFrame
- Se crea el Data Frame con los datos limpios extraidos de los archivos PDF.

In [24]:
#Creación del DataFrame
df = pd.DataFrame(registros)
df.set_index("id", inplace=True)
df.sample(2)

,fuente,arxiv_id,autor,fecha,texto,longitud,titulo,doi
id,,,,,,,,
55,pdf,2509.17924v1,"Xiuqi Ge, Zhibo Yao, Yaosong Du",2025-09-22,M EDICAL P RIORITY F USION : A CHIEVING D UAL ...,10731,Medical priority fusion: achieving dual optimi...,
4,pdf,1804.01770v1,"Saurabh Tiwari, Deepti Ameta, Paramvir Singh, ...",2018-04-05,Teaching Requirements Engineering Concepts usi...,8536,Teaching Requirements Engineering Concepts usi...,10.1145/3194779.3194791


Conversión del DataFrame a CSV
- Se almacena el Data Frame  en un archivo csv

In [25]:
#Convierte el DataFrame a CSV
df.to_csv("articulos_con_metadatos.csv", index=False, encoding="utf-8")

Carga del CSV a Data Frame

- Se lee el archivo
- Se almacena en un Data Frame
- Se despliega una muestra de los datos

In [ ]:
#Lee el CSV para verificar y tomar estadísticas
file = "articulos_con_metadatos.csv"
df = pd.read_csv(file)
df.sample(3)

,fuente,arxiv_id,autor,fecha,texto,longitud,titulo,doi
34,pdf,2509.17647v1,"Yu Liu, Baoxiong Jia, Ruijie Lu, Chuyue Gan, H...",2025-09-22,Preprint V IDEO A RT GS : B UILDING D IGITAL T...,11284,VideoArtGS: Building Digital Twins of Articula...,10.1109/cvpr52688.2022.00553
0,pdf,0402008v1,"David Würfel, Rainer Lutz, Stephan Diehl",2016-7,A USE-CASE DRIVEN APPROACH IN REQUIREMENTS ENG...,3985,A USE-CASE DRIVEN APPROACH IN REQUIREMENTS ENG...,10.1016/j.jss.2015.10.024
50,pdf,2509.17913v2,"Ruixi Huang, David Waxman",2025-09-22,Effective decoupling of mutations and the resu...,31560,Effective decoupling of mutations and the resu...,10.1016/j.jtbi.2025.112277


Descripción de los datos
- Se describe el número de registros
- Se muestran las columnas 
- Se describe el significado de cada columna y los datos que almacena
- Se dan estadísticas de los datos (mínimo, máximo y promedio de palabras)

In [ ]:
#Estadísticas de los datos
print(f"Número de registros: {len(df)}")
print(f"Columnas: {df.columns.tolist()}")
column_descriptions = {
    "id": "Identificador único generado para cada registro.",
    "fuente": "Origen del texto. En este caso, todos provienen de PDFs de arXiv.",
    "arxiv_id": "Identificador único del artículo en arXiv (sin versión).",
    "autor": "Nombre(s) o apellido(s) de los autores principales del artículo.",
    "fecha": "Fecha de publicación o creación del artículo en formato (YYYY-MM-DD).",
    "texto": "Contenido textual completo extraído del PDF, ya limpiado de etiquetas HTML.",
    "longitud": "Número total de palabras contenidas en el campo 'texto'.",
    "titulo": "Título oficial del artículo recuperado desde la API de Crossref (si disponible).",
    "doi": "Identificador persistente DOI del artículo, obtenido de Crossref."
}


print("Descripción de las columnas:\n")
for col, desc in column_descriptions.items():
    print(f"- {col}: {desc}")

#Mínimo, máximo y promedio de longitud de textos
print("\nEstadísticas de longitud de textos:")
print("Mínimo:", df["longitud"].min())
print("Máximo:", df["longitud"].max())
print("Promedio:", round(df["longitud"].mean(), 2))

print("\nPrimeras filas del DataFrame:")
#Primeros 5 registros
df.head()

Número de registros: 200
Columnas: Index(['fuente', 'arxiv_id', 'autor', 'fecha', 'texto', 'longitud', 'titulo',
       'doi'],
      dtype='object')
Descripción de las columnas:

- id: Identificador único generado para cada registro.
- fuente: Origen del texto. En este caso, todos provienen de PDFs de arXiv.
- arxiv_id: Identificador único del artículo en arXiv (sin versión).
- autor: Nombre(s) o apellido(s) de los autores principales del artículo.
- fecha: Fecha de publicación o creación del artículo en formato (YYYY-MM-DD).
- texto: Contenido textual completo extraído del PDF, ya limpiado de etiquetas HTML.
- longitud: Número total de palabras contenidas en el campo 'texto'.
- titulo: Título oficial del artículo recuperado desde la API de Crossref (si disponible).
- doi: Identificador persistente DOI del artículo, obtenido de Crossref.

Estadísticas de longitud de textos:
Mínimo: 3258
Máximo: 95638
Promedio: 13478.0

Primeras filas del DataFrame:


,fuente,arxiv_id,autor,fecha,texto,longitud,titulo,doi
0,pdf,0402008v1,"David Würfel, Rainer Lutz, Stephan Diehl",2016-7,A USE-CASE DRIVEN APPROACH IN REQUIREMENTS ENG...,3985,A USE-CASE DRIVEN APPROACH IN REQUIREMENTS ENG...,10.1016/j.jss.2015.10.024
1,pdf,1012.2469v1,"Daniel Amyot, Ali Echihabi, Yong He",2010-12-11,UCME XPORTER : Supporting scenario transformat...,6564,UCMExporter: Supporting Scenario Transformatio...,10.1007/3-540-48213-x_17
2,pdf,1101.5341v2,"Sergio España, Arturo González, Óscar Pastor, ...",2011-01-27,Informe Técnico / Technical Report Ref. #: Pro...,11943,A practical guide to Message Structures: a mod...,NaN
3,pdf,1804.01770v1,"Saurabh Tiwari, Deepti Ameta, Paramvir Singh, ...",2018-04-05,Teaching Requirements Engineering Concepts usi...,8536,Teaching Requirements Engineering Concepts usi...,10.1145/3194779.3194791
4,pdf,1808.05209v1,"Jin L. C. Guo, Natawut Monaikul, Jane Cleland-...",2018-08-15,Domain Knowledge Discovery Guided by Software ...,6044,Domain Knowledge Discovery Guided by Software ...,10.1109/aire.2018.00006
